In [1]:
import os
from typing import Dict, List

Part 3 - Graph Analytics with Cypher and Neo4j
==============================================

<center><img src="images/Neo4j-logo_color.png" width="500px" /></center>

<br />

In Part 3 of our course, we will use Neo4j and Bloom to learn Graph Analytics against the Open Sanctions database as outlined on Github at [opensanctions/offshore-graph](https://github.com/opensanctions/offshore-graph#sanctionsoffshores-graph-demo).

The following resources will assist you in learning Cypher to query Neo4j and other graph databases via [openCypher](https://opencypher.org/) ([github](https://github.com/opencypher)):

* [Neo4j Cypher Manual - Overview](https://neo4j.com/docs/cypher-manual/current/introduction/cypher_overview/)
* [Neo4j Cypher Manual - Cypher and Neo4j](https://neo4j.com/docs/cypher-manual/current/introduction/cypher_neo4j/)
* [Neo4j 5 Cypher Cheat Sheet](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise/)
* [Neo4j Graph Academy Cypher Fundamentals - 1 Hour Video](https://graphacademy.neo4j.com/courses/cypher-fundamentals/)
* [Neo4j Cypher Manual PDF](https://neo4j.com/docs/pdf/neo4j-cypher-manual-5.pdf)
* [Neo4j Getting Started - Query a Neo4j database using Cypher](https://neo4j.com/docs/getting-started/cypher-intro/)
* [Bite-Sized Neo4j for Data Scientists](https://neo4j.com/video/bite-sized-neo4j-for-data-scientists/) [[github code](https://github.com/cj2001/bite_sized_data_science)]
* [Using Neo4j from Python](https://neo4j.com/developer/python/)

We are going to call Neo4j from Python in this notebook. For simplicity's sake in setting up the course's software, I build a docker image and we run Neo4j Community on Docker via our [docker-compose.yml](docker-compose.yml) file. Docker only supports Neo4j Community unless you get a license (checkout [Neo4j for Startups](https://neo4j.com/startups/)) If you install [Neo4j Desktop](https://neo4j.com/download/) - which comes with a [developers' license]() to [Neo4j Enterprise]() - you can use [Neo4j Bloom](https://neo4j.com/product/bloom/). It is snazzy! :) It can visualize schemas and query results.

# Connecting to Neo4j from Python

We are going to be using two ways to query Cypher.

1) The [neo4j]() PyPi module [[github](https://github.com/neo4j/neo4j-python-driver)], [[Neo4j docs](https://neo4j.com/docs/api/python-driver/current/)]
2) Neo4j's Graphistry Integration [[example notebook](https://github.com/graphistry/pygraphistry/blob/master/demos/demos_databases_apis/neo4j/official/graphistry_bolt_tutorial_public.ipynb)]

We're going to use one or the other as the instructor prefers, depending on how important it is to visualize our results in a table, chart or as a network visualizaton using [Graphistry](https://graphistry.com).

## Connecting Using `neo4j` PyPi Package

The documentation for this library can be out of date, so I'd recommend consulting the source code on Github at [neo4j/neo4j-python-driver](https://github.com/neo4j/neo4j-python-driver) directly, if you run into trouble. I could not import a `neo4j.GraphDatabase` object... kind of an oversight :) It happens, enterprise software is difficult.

This setup uses Docker, which publishes port 7687 to your local machine. Check out the `neo4j` service defined in [docker-compose.yml](docker-compose.yml) if you're curious.

In [21]:
from neo4j import GraphDatabase, RoutingControl

# Docker sends port 7687 to localhost
URI = "neo4j://heracles:7687"
# We have no authentication setup... in practice you may ;)
# AUTH = ("neo4j", "neo4j")

driver = GraphDatabase.driver(URI)
driver

In [ ]:
eager_result = driver.execute_query(
    "MATCH (n) RETURN DISTINCT labels(n), COUNT(*) as c ORDER BY c DESC",
    database_="neo4j",
    routing_=RoutingControl.READ,
)
eager_result

## Using Graphistry via `Plotter.cypher()`



In [5]:
import graphistry

In [6]:
# Environment variable setup
GRAPHISTRY_USERNAME = os.getenv("GRAPHISTRY_USERNAME")
GRAPHISTRY_PASSWORD = os.getenv("GRAPHISTRY_PASSWORD")

In [7]:
graphistry.register(
    api=3,
    username=GRAPHISTRY_USERNAME,
    password=GRAPHISTRY_PASSWORD,
)

In [8]:
# Configuration for Graphistry
GRAPHISTRY_PARAMS = {
    "play": 1000,
    "pointOpacity": 0.7,
    "edgeOpacity": 0.3,
    "edgeCurvature": 0.3,
    "showArrows": True,
    "gravity": 0.15,
    "showPointsOfInterestLabel": False,
    "labels": {
        "shortenLabels": False,
    },
}
FAVICON_URL = "https://graphlet.ai/assets/icons/favicon.ico"
LOGO = {"url": "https://graphlet.ai/assets/Branding/Graphlet%20AI.svg", "dimensions": {"maxWidth": 100, "maxHeight": 100}}

In [20]:
NEO4J = {
    'uri': "bolt://heracles:7687", 
    #'auth': ("neo4j", "myalphapwd1")
}

graphistry.register(bolt=NEO4J)
graphistry.cypher("MATCH (n) RETURN DISTINCT labels(n), COUNT(*) as c ORDER BY c DESC")

/opt/conda/lib/python3.10/site-packages/graphistry/util.py:266: RuntimeWarning: Query returned no edges; may have surprising visual results or need to add missing columns for encodings
  warnings.warn(RuntimeWarning(msg))
/opt/conda/lib/python3.10/site-packages/graphistry/util.py:266: RuntimeWarning: Query returned no nodes
  warnings.warn(RuntimeWarning(msg))


{'bindings': {'edges': Empty DataFrame
  Columns: [_bolt_relationship_id, type, _bolt_start_node_id_key, _bolt_end_node_id_key]
  Index: [],
  'nodes': Empty DataFrame
  Columns: [_bolt_node_id_key, type]
  Index: [],
  'source': '_bolt_start_node_id_key',
  'destination': '_bolt_end_node_id_key',
  'node': '_bolt_node_id_key',
  'edge_label': None,
  'edge_color': None,
  'edge_size': None,
  'edge_weight': None,
  'edge_title': None,
  'edge_icon': None,
  'edge_opacity': None,
  'edge_source_color': None,
  'edge_destination_color': None,
  'point_label': None,
  'point_color': None,
  'point_size': None,
  'point_weight': None,
  'point_title': None,
  'point_icon': None,
  'point_opacity': None,
  'point_x': None,
  'point_y': None},
 'settings': {'height': 500, 'url_params': {'info': 'true'}}}

## Describe Our Network

When you start working with a new graph database, you need to get oriented by describing the schema, then the network, then sampling some data and finally with some exploratory data analysis. From there you know enough to begin your own workflows.

### Describing our Property Graph Model

Let's start by describing our network. What kinds of nodes are there?

In [ ]:
"MATCH (n) RETURN DISTINCT labels(n), COUNT(*) as c ORDER BY c DESC"